### 원문 벡터 db + QA 벡터 DB 하이브리드 (하나의 프롬프트에 같이 주는 방식)에서, TOK_K 변경



In [1]:
from pathlib import Path
import sys

root = Path.cwd()
while not (root / "utils_test").exists() and root != root.parent:
    root = root.parent
sys.path.insert(0, str(root))

from utils_test.main3 import (run_langraph)

C:\Users\playdata\Documents\1team_ai_f\LLM_hybrid\utils_test\retriever.py:51: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs = Chroma(


In [2]:
from utils_test.langgraph_setting2 import graph_setting

graph = graph_setting()

In [3]:
response = run_langraph('Firebase의 updateTime 필드에 사용되는 시간 형식이 뭐야?', '1', None)
print(response)

run_langraph 호출 - 입력: Firebase의 updateTime 필드에 사용되는 시간 형식이 뭐야?, 이미지: False
analyze_image 호출됨 - 이미지 존재: False
검색 중: 'Firebase의 updateTime 필드에 사용되는 시간 형식은 무엇인가요?' with filters: {'tags': {'$in': ['firebase_firestore_crawled']}}
[check_answer_quality] 평가 결과: good
그래프 실행 결과: {'question': 'Firebase의 updateTime 필드에 사용되는 시간 형식이 뭐야?', 'answer': 'Firebase의 `update_time` 필드는 타임스탬프 형식으로 제공됩니다. 이 필드는 데이터베이스 리소스가 가장 최근에 업데이트된 시간을 나타내며, 데이터베이스에 포함된 데이터의 업데이트는 포함하지 않습니다.', 'rewritten': [{'role': 'user', 'content': 'Firebase의 updateTime 필드에 사용되는 시간 형식이 뭐야?'}], 'queries': ['Firebase의 updateTime 필드에 사용되는 시간 형식은 무엇인가요?'], 'search_results': ['웹 버전 9\nimport { doc, setDoc, Timestamp } from "firebase/firestore";\n\nconst docData = {\nstringExample: "Hello world!",\nbooleanExample: true,\nnumberExample: 3.14159265,\ndateExample: Timestamp.fromDate(new Date("December 10, 1815")),\narrayExample: [5, true, "hello"],\nnullExample: null,\nobjectExample: {\na: 5,\nb: {\nnested: "foo"\n}\n}\n};\nawait setDoc(doc(db,

In [7]:
# print(response['answer'])
print(response['tool_calls'])
# print(response['qa_tool_calls'])

[{'tool': 'vector_search_tool', 'args': {'query': 'Firebase의 updateTime 필드에 사용되는 시간 형식은 무엇인가요?', 'api_tags': ['firebase_firestore_crawled']}, 'result': ['웹 버전 9\nimport { doc, setDoc, Timestamp } from "firebase/firestore";\n\nconst docData = {\nstringExample: "Hello world!",\nbooleanExample: true,\nnumberExample: 3.14159265,\ndateExample: Timestamp.fromDate(new Date("December 10, 1815")),\narrayExample: [5, true, "hello"],\nnullExample: null,\nobjectExample: {\na: 5,\nb: {\nnested: "foo"\n}\n}\n};\nawait setDoc(doc(db, "data", "one"), docData);data_types.js [https://github.com/firebase/snippets-web/blob/467eaa165dcbd9b3ab15711e76fa52237ba37f8b/snippets/firestore-next/test-firestore/data_types.js#L8-L24]\n\n웹 버전 8', 'Web\nimport { updateDoc, serverTimestamp } from "firebase/firestore";\n\nconst docRef = doc(db, \'objects\', \'some-id\');\n\n// Update the timestamp field with the value from the server\nconst updateTimestamp = await updateDoc(docRef, {\ntimestamp: serverTimestamp()\n});up

In [9]:
## 환경설정
!pip install -qU ragas langchain langchain-community langchain-openai langchain-chroma langchain-text-splitters pypdf rapidfuzz langgraph gdown chromadb langchain-huggingface sentence-transformers -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.18.0 requires aiofiles<24.0,>=22.0, but you have aiofiles 24.1.0 which is incompatible.
gradio 5.18.0 requires gradio-client==1.7.2, but you have gradio-client 1.10.1 which is incompatible.
gradio 5.18.0 requires markupsafe~=2.0, but you have markupsafe 3.0.2 which is incompatible.


In [2]:
!pip install ragas

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
# 필요한 라이브러리 임포트
import os
import pandas as pd
from pprint import pprint
# LangChain 관련 라이브러리
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# RAGAS 관련 라이브러리
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.persona import Persona
from ragas.testset import TestsetGenerator
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

## 이미 생성된 합성 데이터셋 생성

In [4]:
import pandas as pd

# 같은 폴더(현재 작업 디렉토리)에 저장했다면
dataset_df = pd.read_csv("./dataset.csv")
print(dataset_df.shape)
display(dataset_df)

(20, 3)


,user_input,reference_contexts,reference
0,Analytics Hub 게시자와 목록 관리자 역할의 차이점은 무엇인가요?,사용자가 다음 작업을 수행할 수 있게 해주는 Analytics Hub 게시자 역할(...,"Analytics Hub 게시자 역할은 목록을 생성, 업데이트, 삭제하고 목록에 I..."
1,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 어떻게 해...,BigQuery Studio 저장된 쿼리에 공개 액세스 권한이 부여되지 않도록 하려...,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 프로젝트에...
2,calendarId에 primary 키워드를 사용하면 어떤 캘린더에 접근할 수 있나요?,현재 로그인한 사용자의 기본 캘린더에 액세스하려면 'primary' 키워드를 사용하세요.,calendarId에 primary 키워드를 사용하면 현재 로그인한 사용자의 기본 ...
3,Google Drive API v3의 서비스 엔드포인트가 궁금해,"이 서비스에는 다음 서비스 엔드포인트가 포함되고, 그 아래의 모든 URI가 이 서비...",Google Drive API v3의 서비스 엔드포인트는 https://www.go...
4,files.create 메서드의 최대 파일 크기와 허용되는 미디어 MIME 유형은 ...,"최대 파일 크기: 5,120GB\r\n허용되는 미디어 MIME 유형: */*\r\n...","files.create 메서드의 최대 파일 크기는 5,120GB입니다. 허용되는 미..."
5,Firebase scrypt 해시 비밀번호를 사용한 사용자 가져오는 방법은?,기본적으로 Firebase는 scrypt 해싱 알고리즘의 수정된 Firebase 버...,"Firebase scrypt 해시 비밀번호를 사용한 사용자를 가져오려면, 먼저 Fi..."
6,커스텀 인증 시스템으로 Firebase에 인증하려면 어떻게 해야 하나요?,Firebase 인증에 맞춤 인증 시스템을 통합할 수 있습니다. 사용자가 정상적으로...,"커스텀 인증 시스템으로 Firebase에 인증하려면, 먼저 인증 서버에서 사용자의 ..."
7,"Firestore에서 맵 필드에 대한 단일 필드 색인 예외를 설정하면, 하위 필드들...",맵 필드에 단일 필드 색인 예외를 만들면 맵의 하위 필드에서 해당 설정을 상속합니다...,"네, Firestore에서 맵 필드에 단일 필드 색인 예외를 만들면 맵의 하위 필드..."
8,Firebase의 updateTime 필드에 사용되는 시간 형식이 뭐야?,"Uses RFC 3339, where generated output will alw...",Firebase의 updateTime 필드에는 RFC 3339 시간 형식이 사용됩니...
9,Firefox 58 환경에서 Gmail의 AMP 렌더링 기능이 호환되나요?,브라우저 최소 버전\r\nChrome 69\r\nEdge 지원되지 않음\r\nFir...,"네, Firefox 58 환경에서는 Gmail의 AMP 렌더링 기능이 호환됩니다. ..."


In [5]:

## 02.RAG 체인 구성

import pandas as pd
from datasets import Dataset
import uuid
import time


evaluated_dataset = []

for _, row in dataset_df.iterrows():
    time.sleep(3)
    query = row.user_input  # 사용자 입력 - 합성 데이터셋에서 가져옴

    # run_langraph 함수 호출
    rag_result = run_langraph(query,'1', None)

    evaluated_dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [doc for doc in rag_result["search_results"]],
            "response": rag_result["answer"],
            "reference": row['reference']
        }
    )

print("평가 데이터셋 준비 완료")

# RAGAS 평가 데이터셋 생성
ragas_evaluated_dataset = Dataset.from_list(evaluated_dataset)
# csv 데이터 저장
ragas_evaluated_dataset.to_pandas().to_csv('ragas_evaluated_dataset_hybrid3.csv', index=False)
ragas_evaluated_dataset.to_pandas().info()

run_langraph 호출 - 입력: Analytics Hub 게시자와 목록 관리자 역할의 차이점은 무엇인가요?, 이미지: False
analyze_image 호출됨 - 이미지 존재: False
검색 중: 'Analytics Hub에서 게시자와 목록 관리자 역할의 차이점은 무엇인가요?' with filters: {'tags': {'$in': ['bigquery']}}
[check_answer_quality] 평가 결과: good
그래프 실행 결과: {'question': 'Analytics Hub 게시자와 목록 관리자 역할의 차이점은 무엇인가요?', 'answer': 'Analytics Hub 게시자 역할과 목록 관리자 역할의 차이점은 다음과 같습니다:\n\n1. **Analytics Hub 게시자 역할 (roles/analyticshub.publisher)**:\n   - 사용자가 목록을 생성, 업데이트, 삭제할 수 있습니다.\n   - 목록에 IAM 정책을 설정할 수 있습니다.\n   - 이 역할을 가진 사용자는 "게시자"라고 합니다.\n\n2. **Analytics Hub 목록 관리자 역할 (roles/analyticshub.listingAdmin)**:\n   - 사용자가 목록을 업데이트하고 삭제할 수 있습니다.\n   - 목록에 IAM 정책을 설정할 수 있습니다.\n   - 이 역할을 가진 사용자는 "목록 관리자"라고 합니다.\n\n주요 차이점은 게시자 역할은 목록을 생성할 수 있는 반면, 목록 관리자 역할은 목록을 생성할 수 없다는 점입니다.', 'rewritten': [{'role': 'user', 'content': 'Analytics Hub 게시자와 목록 관리자 역할의 차이점은 무엇인가요?'}], 'queries': ['Analytics Hub에서 게시자와 목록 관리자 역할의 차이점은 무엇인가요?'], 'search_results': ['데이터 교환을 생성, 업데이트, 삭제합니다.\n목록을 생성, 업데이트, 삭제, 공유\nAnalytics H

In [6]:
### RAGAS 평가 실행

# LLM 래퍼 생성
evaluator_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
evaluator_llm = LangchainLLMWrapper(evaluator_llm)

# 평가 메트릭 정의
metrics = [
    LLMContextRecall(),  # 검색된 컨텍스트의 회수율
    Faithfulness(),  # 생성된 답변의 충실도
    FactualCorrectness()  # 사실적 정확성
]
# 평가 실행

result = evaluate(
    dataset=ragas_evaluated_dataset,
    metrics=metrics,
    llm=evaluator_llm,
)

print(result)
# 결과를 DataFrame으로 변환
result_df = result.to_pandas()

# 결과 저장
result_df.to_csv('ragas_evaluation_result5.csv', index=False)

display(result_df.head())
result_df[['context_recall', 'faithfulness', 'factual_correctness(mode=f1)']].describe()

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

{'context_recall': 0.9400, 'faithfulness': 0.9639, 'factual_correctness(mode=f1)': 0.6110}


,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,Analytics Hub 게시자와 목록 관리자 역할의 차이점은 무엇인가요?,"[데이터 교환을 생성, 업데이트, 삭제합니다.\n목록을 생성, 업데이트, 삭제, 공...",Analytics Hub 게시자 역할과 목록 관리자 역할의 차이점은 다음과 같습니다...,"Analytics Hub 게시자 역할은 목록을 생성, 업데이트, 삭제하고 목록에 I...",1.0,1.0,0.92
1,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 어떻게 해...,[탐색기 창에서 프로젝트와 쿼리 폴더를 펼치고 필요한 경우 공유 쿼리 폴더를 펼칩니...,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 프로젝트에...,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 프로젝트에...,1.0,1.0,0.91
2,calendarId에 primary 키워드를 사용하면 어떤 캘린더에 접근할 수 있나요?,[참고: 승인 [https://developers.google.com/workspa...,calendarId에 'primary' 키워드를 사용하면 현재 로그인한 사용자의 기...,calendarId에 primary 키워드를 사용하면 현재 로그인한 사용자의 기본 ...,1.0,1.0,1.00
3,Google Drive API v3의 서비스 엔드포인트가 궁금해,[https://www.googleapis.com/discovery/v1/apis/...,Google Drive API v3의 서비스 엔드포인트는 `https://www.g...,Google Drive API v3의 서비스 엔드포인트는 https://www.go...,1.0,1.0,0.67
4,files.create 메서드의 최대 파일 크기와 허용되는 미디어 MIME 유형은 ...,[이 페이지는 Cloud Translation API [https://cloud.g...,"files.create 메서드의 최대 파일 크기는 5,120GB입니다. 허용되는 미...","files.create 메서드의 최대 파일 크기는 5,120GB입니다. 허용되는 미...",1.0,1.0,0.86


,context_recall,faithfulness,factual_correctness(mode=f1)
count,20.000000,20.000000,20.000000
mean,0.940000,0.963889,0.611000
std,0.225715,0.088348,0.301817
min,0.000000,0.750000,0.000000
25%,1.000000,1.000000,0.415000
50%,1.000000,1.000000,0.715000
75%,1.000000,1.000000,0.815000
max,1.000000,1.000000,1.000000


In [7]:
display(result_df)

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,Analytics Hub 게시자와 목록 관리자 역할의 차이점은 무엇인가요?,"[데이터 교환을 생성, 업데이트, 삭제합니다.\n목록을 생성, 업데이트, 삭제, 공...",Analytics Hub 게시자 역할과 목록 관리자 역할의 차이점은 다음과 같습니다...,"Analytics Hub 게시자 역할은 목록을 생성, 업데이트, 삭제하고 목록에 I...",1.0,1.000000,0.92
1,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 어떻게 해...,[탐색기 창에서 프로젝트와 쿼리 폴더를 펼치고 필요한 경우 공유 쿼리 폴더를 펼칩니...,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 프로젝트에...,BigQuery Studio 저장된 쿼리에 공개 액세스 권한을 방지하려면 프로젝트에...,1.0,1.000000,0.91
2,calendarId에 primary 키워드를 사용하면 어떤 캘린더에 접근할 수 있나요?,[참고: 승인 [https://developers.google.com/workspa...,현재 로그인한 사용자의 기본 캘린더에 접근하기 위해서는 `primary` 키워드를 ...,calendarId에 primary 키워드를 사용하면 현재 로그인한 사용자의 기본 ...,1.0,1.000000,1.00
3,Google Drive API v3의 서비스 엔드포인트가 궁금해,[https://www.googleapis.com/discovery/v1/apis/...,Google Drive API v3의 서비스 엔드포인트는 `https://www.g...,Google Drive API v3의 서비스 엔드포인트는 https://www.go...,1.0,1.000000,0.00
4,files.create 메서드의 최대 파일 크기와 허용되는 미디어 MIME 유형은 ...,[이 페이지는 Cloud Translation API [https://cloud.g...,files.create 메서드를 사용하여 파일을 업로드할 때 지원되는 최대 파일 크...,"files.create 메서드의 최대 파일 크기는 5,120GB입니다. 허용되는 미...",1.0,1.000000,0.86
5,Firebase scrypt 해시 비밀번호를 사용한 사용자 가져오는 방법은?,[Source URL: https://firebase.google.com/docs/...,Firebase scrypt 해시 비밀번호를 사용한 사용자를 가져오는 방법은 다음과...,"Firebase scrypt 해시 비밀번호를 사용한 사용자를 가져오려면, 먼저 Fi...",1.0,1.000000,0.73
6,커스텀 인증 시스템으로 Firebase에 인증하려면 어떻게 해야 하나요?,[Source URL: https://firebase.google.com/docs/...,Firebase에 커스텀 인증 시스템을 통합하려면 다음과 같은 단계를 수행해야 합니...,"커스텀 인증 시스템으로 Firebase에 인증하려면, 먼저 인증 서버에서 사용자의 ...",1.0,0.866667,0.73
7,"Firestore에서 맵 필드에 대한 단일 필드 색인 예외를 설정하면, 하위 필드들...",[맵 필드에 단일 필드 색인 예외를 만들면 맵의 하위 필드에서 해당 설정을 상속합니...,"네, Firestore에서 맵 필드에 단일 필드 색인 예외를 설정하면, 맵의 하위 ...","네, Firestore에서 맵 필드에 단일 필드 색인 예외를 만들면 맵의 하위 필드...",1.0,1.000000,0.75
8,Firebase의 updateTime 필드에 사용되는 시간 형식이 뭐야?,"[웹 버전 9\nimport { doc, setDoc, Timestamp } fro...",Firebase의 updateTime 필드에 사용되는 시간 형식은 RFC 3339 ...,Firebase의 updateTime 필드에는 RFC 3339 시간 형식이 사용됩니...,1.0,1.000000,0.44
9,Firefox 58 환경에서 Gmail의 AMP 렌더링 기능이 호환되나요?,[Source URL: https://developers.google.com/wor...,"네, Firefox 58 환경에서는 Gmail의 AMP 렌더링 기능이 호환됩니다. ...","네, Firefox 58 환경에서는 Gmail의 AMP 렌더링 기능이 호환됩니다. ...",1.0,1.000000,0.50
